In [1]:
import pandas as pd
import urllib3
from bs4 import BeautifulSoup
import requests
import pandas as pd
import csv
import numpy as np

In [2]:
url='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'


In [3]:
respond = requests.get(url)
soup = BeautifulSoup(respond.content, "html.parser")  # lay tao bo du lieu tu url
table = soup.find('table')  # Tim table
rows = table.findAll('tr')

df = pd.DataFrame()
cot0 = []
cot1 = []
cot2 = []

for row in rows[1:]: # Tim tung hang
    cells = row.findAll('td')
    cot0.append(cells[0].find(text=True))
    cot1.append(cells[1].find(text=True))
    cot2.append(cells[2].find(text=True))
df['PostalCode']=cot0
df['Borough']=cot1
df['Neighborhood']=cot2
df.head()

,PostalCode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


In [4]:
df = df[df['Borough'] != 'Not assigned']
for i in range(len(df['Neighborhood'])):
    if df.iloc[i]['Neighborhood'] == 'Not assigned':
        df.iloc[i]['Neighborhood'] = df.iloc[i]['Borough']
df = df.reset_index()
df = df.drop(columns=['index'])
df.head()

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M5A,Downtown Toronto,Regent Park
4,M6A,North York,Lawrence Heights


In [5]:
print (df[:20])

   PostalCode           Borough      Neighborhood
0         M3A        North York         Parkwoods
1         M4A        North York  Victoria Village
2         M5A  Downtown Toronto      Harbourfront
3         M5A  Downtown Toronto       Regent Park
4         M6A        North York  Lawrence Heights
5         M6A        North York    Lawrence Manor
6         M7A      Queen's Park     Not assigned

7         M9A         Etobicoke  Islington Avenue
8         M1B       Scarborough             Rouge
9         M1B       Scarborough           Malvern
10        M3B        North York  Don Mills North

11        M4B         East York  Woodbine Gardens
12        M4B         East York     Parkview Hill
13        M5B  Downtown Toronto           Ryerson
14        M5B  Downtown Toronto  Garden District

15        M6B        North York         Glencairn
16        M9B         Etobicoke       Cloverdale

17        M9B         Etobicoke         Islington
18        M9B         Etobicoke     Martin Grove



In [6]:
dfgb = df.groupby('PostalCode').count()

In [7]:
postal_list = dfgb.index.values.tolist()

In [8]:
c=[]
for postal in postal_list:
    for i in range(len(df)):
        if df.iloc[i]['PostalCode'] == postal:
            a= df.iloc[i]['Borough']
    c.append(a)

In [9]:
b=[]
for postal in postal_list:
    a=''
    for i in range(len(df)):
        if df.iloc[i]['PostalCode'] == postal:
            
            a += '-' + df.iloc[i]['Neighborhood'].strip('\n')
    b.append(a)


In [10]:
d=[]
for i in b:
    a = i.split('-')
    e= a[1]
    if len(a) >2:
        for x in range(2,len(a)):
            e+= ', ' + a[x]
    d.append(e)


In [11]:
dfn = pd.DataFrame()
dfn['PostalCode'] = postal_list
dfn['Borough']=c
dfn['Neighborhood'] = d

print (dfn[:15])

   PostalCode      Borough                                       Neighborhood
0         M1B  Scarborough                                     Rouge, Malvern
1         M1C  Scarborough             Highland Creek, Rouge Hill, Port Union
2         M1E  Scarborough                  Guildwood, Morningside, West Hill
3         M1G  Scarborough                                             Woburn
4         M1H  Scarborough                                          Cedarbrae
5         M1J  Scarborough                                Scarborough Village
6         M1K  Scarborough        East Birchmount Park, Ionview, Kennedy Park
7         M1L  Scarborough                    Clairlea, Golden Mile, Oakridge
8         M1M  Scarborough    Cliffcrest, Cliffside, Scarborough Village West
9         M1N  Scarborough                        Birch Cliff, Cliffside West
10        M1P  Scarborough  Dorset Park, Scarborough Town Centre, Wexford ...
11        M1R  Scarborough                                  Mary

In [12]:
dfn.shape

(103, 3)

In [13]:
!conda install -c conda-forge geocoder --yes 

Solving environment: done

## Package Plan ##

  environment location: /home/jupyterlab/conda

  added / updated specs: 
    - geocoder


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    orderedset-2.0             |           py36_0         231 KB  conda-forge
    geocoder-1.38.1            |             py_0          52 KB  conda-forge
    ratelim-0.1.6              |           py36_0           5 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         288 KB

The following NEW packages will be INSTALLED:

    geocoder:   1.38.1-py_0  conda-forge
    orderedset: 2.0-py36_0   conda-forge
    ratelim:    0.1.6-py36_0 conda-forge


orderedset-2.0       | 231 KB    | ##################################### | 100% 
geocoder-1.38.1      | 52 KB     | ##################################### | 100% 
ratelim-0.1.6      

In [14]:
import geocoder

In [15]:
latitude = []
longitude = []

for y in dfn['PostalCode']:
    lat_lng_coords = None
    while(lat_lng_coords is None):
        g = geocoder.google('{}, Toronto, Ontario'.format(y), key='')
        lat_lng_coords = g.latlng

    latitude.append(lat_lng_coords[0])
    longitude.append(lat_lng_coords[1])
dfn['Latitude'] = latitude
dfn['Longitude'] = longitude
dfn.head()

Status code Unknown from https://maps.googleapis.com/maps/api/geocode/json: ERROR - ('Connection aborted.', OSError("(104, 'ECONNRESET')",))


,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


In [16]:
dfn.shape

(103, 5)

In [17]:
df_to = dfn[dfn['Borough'].str.contains('Toronto')]
df_to.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
37,M4E,East Toronto,The Beaches,43.676357,-79.293031
41,M4K,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188
42,M4L,East Toronto,"The Beaches West, India Bazaar",43.668999,-79.315572
43,M4M,East Toronto,Studio District,43.659526,-79.340923
44,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790


In [18]:
CLIENT_ID = 'WB2MMTXYA1ROXPDLSVX5XCJCFLEVOI1QEWQ2PGTFGSXSJS1Z' # your Foursquare ID
CLIENT_SECRET = 'Q143TFXFSJQGQ1B225UGAUITQ0P1A3UVLVUC15ARGTESUB5V' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
radius = 500
LIMIT = 100

In [19]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [20]:
toronto_venues = getNearbyVenues(names=df_to['Neighborhood'],
                                 latitudes=df_to['Latitude'],
                                 longitudes=df_to['Longitude'])

The Beaches
The Danforth West, Riverdale
The Beaches West, India Bazaar
Studio District
Lawrence Park
Davisville North
North Toronto West
Davisville
Moore Park, Summerhill East
Deer Park, Forest Hill SE, Rathnelly, South Hill, Summerhill West
Rosedale
Cabbagetown, St. James Town
Church and Wellesley
Harbourfront, Regent Park
Ryerson, Garden District
St. James Town
Berczy Park
Central Bay Street
Adelaide, King, Richmond
Harbourfront East, Toronto Islands, Union Station
Design Exchange, Toronto Dominion Centre
Commerce Court, Victoria Hotel
Roselawn
Forest Hill North, Forest Hill West
The Annex, North Midtown, Yorkville
Harbord, University of Toronto
Chinatown, Grange Park, Kensington Market
CN Tower, Bathurst Quay, Island airport, Harbourfront West, King and Spadina, Railway Lands, South Niagara
Stn A PO Boxes 25 The Esplanade
First Canadian Place, Underground city
Christie
Dovercourt Village, Dufferin
Little Portugal, Trinity
Brockton, Exhibition Place, Parkdale Village
High Park, The 

In [21]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 232 uniques categories.


In [24]:
from sklearn.cluster import KMeans

df_to_kmeans = df_to.drop(columns=['PostalCode','Borough','Neighborhood'])
kclusters = 5
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(df_to_kmeans)
df_to['label']=kmeans.labels_


/home/jupyterlab/conda/lib/python3.6/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [28]:
df_to.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude,label
37,M4E,East Toronto,The Beaches,43.676357,-79.293031,3
41,M4K,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188,3
42,M4L,East Toronto,"The Beaches West, India Bazaar",43.668999,-79.315572,3
43,M4M,East Toronto,Studio District,43.659526,-79.340923,3
44,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790,1


In [22]:
!conda install -c conda-forge folium=0.5.0 --yes
import folium # plotting library

Solving environment: done

# All requested packages already installed.



In [ ]:

import matplotlib.cm as cm
import matplotlib.colors as colors

# create map
map_clusters = folium.Map(location=[43.676357,-79.293031], zoom_start=11)
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]
# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(df_to['Latitude'], df_to['Longitude'], df_to['Neighborhood'], df_to['labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters